# edit anywhere

Purpose: change the web however you want, save and restore it from github, treat the web like one giant web of gists.



## introduction

First attempt at this app was using megamind.bot app, the editor takes an input URL, and a filename.


 It loads a website from crawled data from Selenium/data collection.ipynb. It then sent these parameters to the client app for displaying in the same frame.

The new implementation should act more like a marketing website. Enter a URL in a box in the middle like Google, use the controls that appear on the copied page content. Use different methods to get the page content such as Selenium crawler or simple phantom browser.

After the content is crawled, make any changes that save to a single Gist for the entire domain. Load the gist from domain when the page is loaded (plugin phase 2).

TODO:

Fix SPA/PWA apps and canvas copies.

Move page processing to generalized Selenium, convert scripts.

TODO: Use print to PDF in chrome ignoring print styles from Developer mode, better Selenium crawler.

TODO: Make a tool for the page manipulation, this is a common theme.


## gist

Read and write files from gist.



### read gist files


#### the code

read gist files?


In [ ]:
var Octokit = require('@octokit/rest');

// commit changes to github
function getGist(gist) {
    if(!gist) return {}
    const github = new Octokit({
        host: 'api.github.com'
    });
    /*
    github.authenticate({
        type: 'basic',
        username: process.env.USERNAME,
        password: process.env.PASSWORD
    });
    */

    //return github.gists.get({gist_id: gist})
    return github.gists.get({gist_id: gist})
        .then(r => r.data)
        .catch(e => console.log(e))
}

module.exports = getGist


#### test gist


In [ ]:
var importer = require('../Core');
var getGist = importer.import('read gist files');

if(typeof $$ !== 'undefined') {
    $$.async();
    getGist('a572d0830ae72b962e12a57adaec7c52')
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


### write gist files

write gist files?


#### the code 



## ckeditor client

Use CKEditor and some scripts to apply some ACLs to the page and output.



### apply ACL


#### the code
apply acl to html?


In [ ]:
// scan using an acl list, similar to easylist?
// TODO: accept formats:
//    {"selector": "selector"}
//    {"glob-url@selector": "glob-template-path@selector"}
//    {"selector": "html-file@selector"}
//    {"selector": "html-file@xpath"} ?
//    {"glob-file": {"glob-url"...} || ["selector"]}
const paths = JSON.parse('[]');

### load ckeditor


#### the code

load ckeditor?


In [ ]:
var {URL} = require('url')
var importer = require('../Core')
var loadScraped = importer.import('get scraped page')
var getGist = importer.import('read gist files')
var {selectDom} = importer.import('select tree')

// git 
async function gitEditor(url, gist) {
    // TODO: use a Github repo as the input
    if(typeof url == 'string') {
        url = new URL(url);
    }
    var file = url.pathname
    var host = url.hostname.replace(/[^a-z0-9_-]/ig, '_')
    if(!file) file = 'index'

    var files = await loadScraped(url)
    if(typeof files[ host + '-acl.json' ] === 'undefined') {
        var saved = (await getGist(gist)).files
        if(saved && saved[host + '-acl.json']) {
            files[host + '-acl.json'] = JSON.parse(saved[host + '-acl.json'].content || '[]')
            if(typeof files[host + '-acl.json'] === 'string') {
                files[host + '-acl.json'] = [files[host + '-acl.json']]
            }
        }
    }

    // inject the editor into copied page
    var doc = selectDom('*', files[file])
    var body = selectDom('//body', doc)
    console.log(body.append)
    if(body) {
        //body.append(selectDom('//script', '<script src="https://cdn.ckeditor.com/ckeditor5/16.0.0/classic/ckeditor.js"></script>'))
        // add content editable to -acl list elements
        (files[host + '-acl.json'] || []).forEach(i => {
            var els = selectDom([i], body)
            els.forEach(el => {
                el.setAttribute('contenteditable', 'contenteditable')
            })
        })
        console.log('done')
        return body.ownerDocument.documentElement.outerHTML
    } else {
        throw Error(`Not found ${url}`)
    }
}

module.exports = gitEditor

if(typeof $$ !== 'undefined') {
    $$.async();
    gitEditor('https://www.google.com')
        .then(r => $$.mime({'text/html': r}))
        .catch(e => $$.sendError(e))
}


## tools


### restrain CSS

Replace all CSS rules with a container ID to restain it's affects on the page.


#### the code

restrain css? 

scope css?



In [ ]:
var css = require('css');

function prefixCssRules(str, prefix) {
    try {
        const ast = css.parse(str);
        // TODO: add a check for media queries
        ast.stylesheet.rules.forEach(r => {
            if(typeof r.selectors === 'undefined') {
                return;
            }
            r.selectors.forEach((s, i) => {
                if(s.includes('body')) {
                    r.selectors[i] = s.replace(/\s*body\s*/ig, prefix);
                } else {
                    r.selectors[i] = prefix + ' ' + s;
                }
            });
        })
        return css.stringify(ast);
    } catch (e) {
        console.log(e)
        return str
    }
    
}

module.exports = prefixCssRules;


### TODO: express crawl middleware

Serve every static address from a cache crawl json file.

TODO: move this to data collection tools


### read crawl files

Load matching files from a crawled cache json file. See Selenium/data collection.ipynb for more information on crawl cache json.

TODO: move this to data collection.ipynb tools


#### the code 

read crawl files?

get scraped page?


In [ ]:
var path = require('path')
var fs = require('fs')
var {URL} = require('url')
var uuid = require('uuid/v1')
var importer = require('../Core')
var {glob} = importer.import('glob files')
var {minimatch} = importer.import('minimatch')
var {selectDom} = importer.import('select tree')
var prefixCssRules = importer.import('scope css')
var {findCache} = importer.import('domain crawler tools')

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = path.join(PROFILE_PATH, 'Collections/crawls');

function matchPage(match, search, hostname) {
    return search.includes(match)
        || minimatch(search, match)
        || (!match || match === 'index')
        && search.match(/https?:\/\/[^\/]*\/?$/ig)
        && search.includes(hostname)
}

function loadScraped(url) {
    if(typeof url == 'string') {
        url = new URL(url);
    }
    //console.log(url)
    var host = url.hostname
    var file = url.pathname
    if(!file) file = 'index'
    
    // lookup on filesystem
    var cache = findCache(host)
    if(!cache[0]) {
        return
    }
    const crawl = JSON.parse(fs.readFileSync(cache[0]).toString());
    const entry = crawl.filter(r => matchPage(file, r.url, host))[0];
    const result = {}
    //console.log(entry)
    // parse out styles and images and package it up in to one nice page
    if(entry) {
        const HOST_ID = host.replace(/[^a-z0-9_-]/ig, '_')
        var doc = selectDom('*', entry.html)
        var styles = selectDom(['//link[@rel = "stylesheet"]|//style'], doc)
        var css = ''
        styles.forEach(s => {
            var src = s.getAttribute('src') || s.getAttribute('href')
            s.remove()
            if(!src) {
                css += s.innerHTML
                return
            }
            src = new URL(src, url).href
            var rules = crawl.filter(r => r.url === src)[0]
            if(rules) {
                css += rules.content
            }
        })
        
        var scripts = selectDom(['//script'], doc)
        scripts.forEach(s => s.remove())
        
        var images = selectDom(['//img'], doc)
        images.forEach(i => {
            var src = i.getAttribute('src')
            src = new URL(src, url).href
            var images = crawl.filter(r => r.url === src)[0]
            if(images && images.content.includes('data:')) {
                i.setAttribute('src', images.content)
            }
        })
        
        var links = selectDom(['//a'], doc)
        links.forEach(l => {
            var src = l.getAttribute('href')
            src = new URL(src, url).href
            l.setAttribute('href', '/?url=' + src)
        })
        
        // TODO: load images as data URIs and lower quality
        css = prefixCssRules(css, '#' + HOST_ID)
        .replace(/url\s*\(['"]*([^\)]*?)['"]*\)/ig, ($0, $1) => {
            var src = new URL($1, url).href
            var images = crawl.filter(r => r.url === src)[0]
            if(images && images.content.includes('data:')) {
                return `url(${images.content})`
            }
            return $0
        })
        .replace(/href="([^\"]*)"/ig, ($0, $1) => {
            var src = new URL($1, url).href
            return $0.replace($1, '/?url=' + src)
        })
        var body = selectDom('//body', doc)
        var classes = body.getAttribute('class')
        result[file] = `<html><head><style>
body, html {
    margin: 0;
    padding: 0;
}
</style><style>${css}</style>
</head><body><div id="${HOST_ID}" class="${classes}">${body.innerHTML}</div>
<script>
var script = document.createElement('script')
script.onload = function () {
    var editors = document.querySelectorAll( '*[contenteditable]' )
    editors.forEach(e => {
        InlineEditor
        .create( e )
        .catch( error => console.error( error ) )
    })
}
script.setAttribute('src', 'https://cdn.ckeditor.com/ckeditor5/16.0.0/inline/ckeditor.js')
document.body.appendChild(script)
</script>
</body></html>`
    }
    return result
}

module.exports = loadScraped

//var importer = require('../Core')
//var loadScraped = importer.import('read crawl files')

if(typeof $$ != 'undefined') {
    var scraped = loadScraped('https://google.com')
    $$.html(scraped)
}


#### test crawl cache loader

